In [ ]:
import pandas as pd
!wget https://dl.google.com/linux/linux_signing_key.pub
!sudo apt-key add linux_signing_key.pub
!echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' >> /etc/apt/sources.list.d/google-chrome.list
!sudo apt-get -y update
!sudo apt-get install -y google-chrome-stable

In [ ]:
# install chromedriver
# !apt-get install -y qq unzip
!wget -O /tmp/chromedriver.zip http://chromedriver.storage.googleapis.com/`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip chromedriver -d /usr/local/bin/

In [ ]:
# install selenium
!sudo apt install -y python3-selenium
!pip install selenium scrapy

In [ ]:
# To check Google Chrome's version
!google-chrome --version

In [ ]:
# To check Chrome Driver's version
!chromedriver -v

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
import time

# Scrape jobs.cz

In [ ]:
def jobscz_selenium(url):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(
        options=chrome_options
    )
    driver.get(url)
    """wait = WebDriverWait(driver, 20)  # Maximum wait time of 10 seconds
    div_element = wait.until(
        EC.visibility_of_element_located((By.CLASS_NAME, "cp-detail__content"))
    )"""
    time.sleep(5)
    div_element = driver.find_element(By.CLASS_NAME, "cp-detail__content")
    text_values = div_element.text
    driver.quit()
    return text_values

In [ ]:
import datetime
import re
import os
import scrapy

In [ ]:
class DataSpider(scrapy.Spider):
    name = "data"
    allowed_domains = ["jobs.cz"]
    start_urls = [
        "https://www.jobs.cz/prace/?q%5B%5D=machine%20learning&q%5B%5D=data%20analytics&q%5B%5D=data%20science&q%5B%5D=MLOps&q%5B%5D=ML&q%5B%5D=strojové%20učení&q%5B%5D=BI%20analytik&q%5B%5D=Buissness%20Analytik&q%5B%5D=bi%20analyst&q%5B%5D=data%20analyst&profession%5B%5D=201100610&profession%5B%5D=201100592&profession%5B%5D=201100418&profession%5B%5D=201100618&profession%5B%5D=201100658"
    ]
            
    def parse(self, response):
        jobs = response.css("article.SearchResultCard")

        for job in jobs:
            relative_url = job.css("h2 a ::attr(href)").get()
            title = job.css("h2 a ::text").get().strip()
            pay_range = job.css("span[class*=Tag--success]::text").get()
            comapny = response.css(
                "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:first-child span::text"
            ).get()
            location = (
                response.css(
                    "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:nth-child(2)"
                )
                .re(r">([^<]+)<")[3]
                .strip()
            )
            yield response.follow(
                relative_url,
                callback=self.parse_inzerat,
                meta={
                    "title": title,
                    "pay_range": pay_range,
                    "comapny": comapny,
                    "location": location,
                },
            )

        """next_page = response.css(
            "ul.Pagination li.Pagination__item:last-child a::attr(href)"
        ).get()
        if next_page:
            next_page_url = self.domain_name + next_page
            yield response.follow(next_page_url, callback=self.parse)"""

    def parse_inzerat(self, response):
        all_text = "Not Found"
        div_content = (
            response.css("section.cp-detail__content")
            or response.css("div.cp-detail__content")
            or response.css("div.content-rich-text")
            or response.css("div.standalone.jobad__body")
        )
        if div_content:
            # Extract text from <p> tags
            paragraphs = div_content.css("p::text").getall()

            # Extract text from <h2> tags
            headings = div_content.css("h2::text").getall()

            # Extract text from <ul> tags
            list_items = div_content.css("ul li::text").getall()

            # Extract text from <strong> tags
            strong_text = div_content.css("strong::text").getall()
            
            i_text = div_content.css("i::text").getall()
            
            u_text = div_content.css("u::text").getall()
            # Join all the extracted text into a single string
            all_text = div_content.select("text()").extract()
        else:
            #all_text = jobscz_selenium(response.url)
            pass

        yield {"url": response.url, "text": all_text, **response.meta, "lists": list_items,
               "paragraphs": paragraphs, strong_text: }

In [ ]:
class WebDevSpider(scrapy.Spider):
    name = "webdev"
    allowed_domains = ["jobs.cz"]
    start_urls = [
        "https://www.jobs.cz/prace/?q%5B%5D=frontend&q%5B%5D=backend&q%5B%5D=full%20stack&q%5B%5D=web%20developer&q%5B%5D=react&q%5B%5D=vue&q%5B%5D=java&q%5B%5D=javascript&q%5B%5D=typescript&q%5B%5D=spring&q%5B%5D=django&q%5B%5D=flask&q%5B%5D=node&q%5B%5D=angular&page=2"
    ]
            
    def parse(self, response):
        jobs = response.css("article.SearchResultCard")

        for job in jobs:
            relative_url = job.css("h2 a ::attr(href)").get()
            title = job.css("h2 a ::text").get().strip()
            pay_range = job.css("span[class*=Tag--success]::text").get()
            comapny = response.css(
                "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:first-child span::text"
            ).get()
            location = (
                response.css(
                    "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:nth-child(2)"
                )
                .re(r">([^<]+)<")[3]
                .strip()
            )
            yield response.follow(
                relative_url,
                callback=self.parse_inzerat,
                meta={
                    "title": title,
                    "pay_range": pay_range,
                    "comapny": comapny,
                    "location": location,
                },
            )

        """next_page = response.css(
            "ul.Pagination li.Pagination__item:last-child a::attr(href)"
        ).get()
        if next_page:
            next_page_url = self.domain_name + next_page
            yield response.follow(next_page_url, callback=self.parse)"""

    def parse_inzerat(self, response):
        all_text = "Not Found"
        div_content = (
            response.css("section.cp-detail__content")
            or response.css("div.cp-detail__content")
            or response.css("div.content-rich-text")
            or response.css("div.standalone.jobad__body")
        )
        if div_content:
            # Extract text from <p> tags
            paragraphs = div_content.css("p::text").getall()

            # Extract text from <h2> tags
            headings = div_content.css("h2::text").getall()

            # Extract text from <ul> tags
            list_items = div_content.css("ul li::text").getall()

            # Extract text from <strong> tags
            strong_text = div_content.css("strong::text").getall()

            # Join all the extracted text into a single string
            all_text = " ".join(
                paragraphs + headings + list_items + strong_text
            )
        else:
            #all_text = jobscz_selenium(response.url)

        yield {"url": response.url, "text": all_text, **response.meta}

In [ ]:
class DevOpsSpider(scrapy.Spider):
    name = "devops"
    allowed_domains = ["jobs.cz"]
    start_urls = [
        "https://www.jobs.cz/prace/?q%5B%5D=devops&q%5B%5D=dataops&q%5B%5D=mlops"
    ]
            
    def parse(self, response):
        jobs = response.css("article.SearchResultCard")

        for job in jobs:
            relative_url = job.css("h2 a ::attr(href)").get()
            title = job.css("h2 a ::text").get().strip()
            pay_range = job.css("span[class*=Tag--success]::text").get()
            comapny = response.css(
                "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:first-child span::text"
            ).get()
            location = (
                response.css(
                    "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:nth-child(2)"
                )
                .re(r">([^<]+)<")[3]
                .strip()
            )
            yield response.follow(
                relative_url,
                callback=self.parse_inzerat,
                meta={
                    "title": title,
                    "pay_range": pay_range,
                    "comapny": comapny,
                    "location": location,
                },
            )

        """next_page = response.css(
            "ul.Pagination li.Pagination__item:last-child a::attr(href)"
        ).get()
        if next_page:
            next_page_url = self.domain_name + next_page
            yield response.follow(next_page_url, callback=self.parse)"""

    def parse_inzerat(self, response):
        all_text = "Not Found"
        div_content = (
            response.css("section.cp-detail__content")
            or response.css("div.cp-detail__content")
            or response.css("div.content-rich-text")
            or response.css("div.standalone.jobad__body")
        )
        if div_content:
            # Extract text from <p> tags
            paragraphs = div_content.css("p::text").getall()

            # Extract text from <h2> tags
            headings = div_content.css("h2::text").getall()

            # Extract text from <ul> tags
            list_items = div_content.css("ul li::text").getall()

            # Extract text from <strong> tags
            strong_text = div_content.css("strong::text").getall()

            # Join all the extracted text into a single string
            all_text = " ".join(
                paragraphs + headings + list_items + strong_text
            )
        else:
            #all_text = jobscz_selenium(response.url)

        yield {"url": response.url, "text": all_text, **response.meta, }

In [ ]:
class CloudSpider(scrapy.Spider):
    name = "cloud"
    allowed_domains = ["jobs.cz"]
    start_urls = [
        "https://www.jobs.cz/prace/?q%5B%5D=cloud&q%5B%5D=cloudu"
    ]
            
    def parse(self, response):
        jobs = response.css("article.SearchResultCard")

        for job in jobs:
            relative_url = job.css("h2 a ::attr(href)").get()
            title = job.css("h2 a ::text").get().strip()
            pay_range = job.css("span[class*=Tag--success]::text").get()
            comapny = response.css(
                "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:first-child span::text"
            ).get()
            location = (
                response.css(
                    "ul.SearchResultCard__footerList li.SearchResultCard__footerItem:nth-child(2)"
                )
                .re(r">([^<]+)<")[3]
                .strip()
            )
            yield response.follow(
                relative_url,
                callback=self.parse_inzerat,
                meta={
                    "title": title,
                    "pay_range": pay_range,
                    "comapny": comapny,
                    "location": location,
                },
            )

        """next_page = response.css(
            "ul.Pagination li.Pagination__item:last-child a::attr(href)"
        ).get()
        if next_page:
            next_page_url = self.domain_name + next_page
            yield response.follow(next_page_url, callback=self.parse)"""

    def parse_inzerat(self, response):
        all_text = "Not Found"
        div_content = (
            response.css("section.cp-detail__content")
            or response.css("div.cp-detail__content")
            or response.css("div.content-rich-text")
            or response.css("div.standalone.jobad__body")
        )
        if div_content:
            # Extract text from <p> tags
            paragraphs = div_content.css("p::text").getall()

            # Extract text from <h2> tags
            headings = div_content.css("h2::text").getall()

            # Extract text from <ul> tags
            list_items = div_content.css("ul li::text").getall()

            # Extract text from <strong> tags
            strong_text = div_content.css("strong::text").getall()

            # Join all the extracted text into a single string
            all_text = " ".join(
                paragraphs + headings + list_items + strong_text
            )
        else:
            #all_text = jobscz_selenium(response.url)

        yield {"url": response.url, "text": all_text, **response.meta}

In [ ]:
!ls

In [ ]:
from scrapy.crawler import CrawlerRunner
from twisted.internet import reactor, defer

runner = CrawlerRunner(settings={
    "FEEDS": {
        "/kaggle/working/jobs_cz.json": {"format": "json"},
    },
})

spiders = [DataSpider, WebDevSpider, DevOpsSpider, CloudSpider]  # Add more spiders as needed

@defer.inlineCallbacks
def crawl():
    for spider in spiders:
        yield runner.crawl(spider)
    reactor.stop()
crawl()
reactor.run()

# Scrape startupjobs.cz

In [ ]:
"""import json
n=1 
postings = []
description = []

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(
    options=chrome_options
)

# Open the webpage
driver.get(
    "https://www.startupjobs.cz/nabidky?superinput=data&lokalita=ChIJEVE_wDqUEkcRsLEUZg-vAAQ,ChIJi3lwCZyTC0cRkEAWZg-vAAQ&lokalita-vzdalene=1"
)  # Replace with the actual URL
driver.save_screenshot("ss.png")
while True:
    try:
        # Find the div element by its ID
        wait = WebDriverWait(driver, 30)  # Maximum wait time of 10 seconds
        div_element = wait.until(
            EC.visibility_of_element_located((By.ID, "offers-list"))
        )

        wait = WebDriverWait(driver, 10) 
        article = wait.until(
            EC.visibility_of_element_located((By.TAG_NAME, "article"))
        )
        # Find all the article elements inside the div
        article_elements = div_element.find_elements(By.TAG_NAME, "article")

        
        article_urls = []
        # Iterate over each article and extract the text
        for article in article_elements:
            atext = article.text
            link_element = article.find_element(By.CSS_SELECTOR, "a")
            article_urls.append(link_element.get_attribute("href"))
            postings.append(atext)

        for url in article_urls:
            driver.get(url)
            wait = WebDriverWait(driver, 10) 
            text_element = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "fr-view")))
            text = text_element.text
            description.append(text)

        n+=1
        break

        driver.get(f"https://www.startupjobs.cz/nabidky/strana-{str(n)}?superinput=data&lokalita=ChIJEVE_wDqUEkcRsLEUZg-vAAQ,ChIJi3lwCZyTC0cRkEAWZg-vAAQ&lokalita-vzdalene=1")
    except Exception as e:
        print(e)
        break


driver.quit()

data = {
    "postings": postings,
    "descriptions": description
}
with open('startup_jobs.json', 'w') as fp:
    json.dump(data, fp)"""

## Clean data

In [ ]:
import json

def process_json_file(file_path):
    with open(file_path, 'r') as file:
        # Read the JSON file as a string
        json_string = file.read()

    # Remove combinations: "\n][\n", "]\n", and "[\n"
    modified_string = json_string.replace('\n][\n', ',\n')

    # Load the modified string as a dictionary
    data = json.loads(modified_string)

    return data

# Example usage
file_path = '/kaggle/working/jobs_cz.json'
json_data = process_json_file(file_path)

In [ ]:
import json
import re
from statistics import median

with open("/kaggle/input/test-scrape/devops_technologies.json", "r", encoding="utf-8") as f:
    devops = json.load(f)
with open("/kaggle/input/test-scrape/data_technologies.json", "r", encoding="utf-8") as f:
    data = json.load(f)
with open("/kaggle/input/test-scrape/web_technologies.json", "r", encoding="utf-8") as f:
    web = json.load(f)


def extract_pay(pay_range):
    cleaned_range = re.sub(r"[^\d\s]", "", pay_range)

    values = cleaned_range.split()

    int_values = [int(val) for val in values if int(val) != 0]

    median_value = median(int_values)

    return int(median_value * 1000)


def get_bigrams(text):
    # Split the text into individual words
    words = text.split()

    # Create a list to store the bigrams
    bigrams = []

    # Iterate through the words to generate bigrams
    for i in range(len(words) - 1):
        # Concatenate adjacent words to form a bigram
        bigram = (words[i], words[i + 1])

        # Append the bigram to the list
        bigrams.append(bigram)

    return bigrams


def keep_alphanumeric_space(text):
    # Use regex to keep only words, digits, and spaces
    cleaned_text = re.sub(r"[^\w\s\d.]", "", text)
    return cleaned_text


ultra_dict = {**data, **devops, **web}
tech = ultra_dict.keys()
new_dict = []
for idx, i in enumerate(json_data):
    i["tech"] = []
    txt_low = (
        i["text"].lower() + " " + keep_alphanumeric_space(i["title"].lower())
    )
    unigrams = txt_low.split(" ")
    bigrams = get_bigrams(txt_low)
    for q in unigrams:
        if q in tech and q not in i["tech"]:
            i["tech"] += [q]
    for q in bigrams:
        if q in tech and q not in i["tech"]:
            i["tech"] += [q]
    new_data = {
        "TITLE": i["title"],
        "COMAPNY": i["comapny"],
        "LOCATION": i["location"],
        "PAY": extract_pay(i["pay_range"]) if i["pay_range"] else "Unknown",
        "REQUIRED_TECH": i["tech"],
        "BASE_TEXT": i["text"],
        "URL": i["url"],
    }
    new_dict.append(new_data)
with open("jobs_cz_docs.json", "w", encoding="utf-8") as f:
    json.dump(new_dict, f, ensure_ascii=False, indent=4)